In [1]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='latin-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [2]:
# Оставляем только интересующие нас колонки — тексты смс и метки:
df = df[['v1', 'v2']]
df = df.rename(columns = {'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Удаляем дублирующиеся тексты:
df = df.drop_duplicates('text')

In [4]:
# Заменяем метки на бинарные:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

### Предобработка текста (Задание 1)

Нужно дополнить функцию для предобработки сообщений, которая делает с текстом следующее:
* приводит текст к нижнему регистру;
* удаляет стоп-слова;
* удаляет знаки препинания;
* нормализует текст при помощи стеммера Snowball.

Предлагаем воспользоваться библиотекой nltk, чтобы не составлять список стоп-слов и не реализовывать алгоритм стемминга самостоятельно. Примеры использования стеммеров можно найти по ссылке (https://www.nltk.org/howto/stem.html).

In [5]:
# Для начала загружу стоп слова через терминал
# >> import nltk
# >> nltk.download("stopwords")
import string
import nltk
from nltk import stem
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer   

stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()  # Приводим к нижнему регистру
    word_ws = text.split()  # Разбиваем текст на слова в списке
    # word_ws=[w.lower() for w in word if w.isalpha()]  
    filtered_sentence=[w for w in word_ws if w not in stop_words]  # Удаляем стоп-слова  
    normilize_sentence=[stemmer.stem(word) for word in filtered_sentence]  # Нормализуем текст при помощи стеммера Snowball
    text = (" ").join(normilize_sentence)  # Вновь соединяем текст въедино
    
    return text
    

In [6]:
text = '''I'm gonna be home soon, and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.'''
x = preprocess(text)





Проверка, что функция работает верно

In [7]:
assert preprocess("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.") == "im gonna home soon dont want talk stuff anymor tonight k ive cri enough today"
assert preprocess("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...") == "go jurong point crazi avail bugi n great world la e buffet cine got amor wat"

Применяем получившуюся функцию к текстам:

In [8]:
df['text'] = df['text'].apply(preprocess)

### Разделение данных на обучающую и тестовую выборки (Задание 2)

In [9]:
y = df['label'].values

Теперь нужно разделить данные на тестовую (test) и обучающую (train) выборку. В библиотеке scikit-learn для этого есть готовые инструменты.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.35, random_state=30)

### Обучение классификатора (Задание 3)

Переходим к обучению классификатора.

Сначала извлекаем признаки из текстов. Рекомендуем попробовать разные способы и посмотреть, как это влияет на результат (подробнее о различных способах представления текстов можно прочитать по ссылке https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction).

Затем обучаем классификатор. Мы используем SVM, но можно поэкспериментировать с различными алгоритмами.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Извлекаем признаки из текстов
vectorizer = TfidfVectorizer(decode_error='ignore')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [12]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Обучаем модель SVM
model = LinearSVC(random_state = 30, C = 1.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

Для самопроверки. Если вы верно дополнили функцию ```preprocess```, то должны получиться следующие результаты оценки модели.

In [13]:
print(classification_report(y_test, predictions, digits=3))

              precision    recall  f1-score   support

           0      0.980     0.996     0.988      1568
           1      0.972     0.868     0.917       242

    accuracy                          0.979      1810
   macro avg      0.976     0.932     0.953      1810
weighted avg      0.979     0.979     0.978      1810



Выполним предсказание для конкретного текста

In [14]:
txt = "Excellent collection of articles and speeches."
txt = preprocess(txt)
txt = vectorizer.transform([txt])

In [15]:
# 0 соответствует ham, 1 -- spam
model.predict(txt)

array([0], dtype=int64)